IMDB 영화평 감성분석(이진분류)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from google.colab import files
# up = files.upload()

In [3]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t')
df.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [4]:
df = pd.read_csv('labeledTrainData.tsv', sep='\t', quoting=3)  # 3: QUOTE_NONE
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         25000 non-null  object
 1   sentiment  25000 non-null  int64 
 2   review     25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [6]:
df.review[0][:1000]

'"With all this stuff going down at the moment with MJ i\'ve started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ\'s feeling towards the press and also the obvious message of drugs are bad m\'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally

- 텍스트 전처리

In [7]:
# <br /> 태그는 공백으로 변환
df.review = df.review.str.replace('<br />', ' ')
df.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [8]:
# 구둣점, 숫자 제거 - 영문자가 아닌 글자는 전부 공백으로 변환
# 데이터 프레임의 str method는 정규식 표현을 지원함
df.review = df.review.str.replace('[^a-zA-Z]', ' ').str.strip()
df.review[0][:1000]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


'With all this stuff going down at the moment with MJ i ve started listening to his music  watching the odd documentary here and there  watched The Wiz and watched Moonwalker again  Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent  Moonwalker is part biography  part feature film which i remember going to see at the cinema when it was originally released  Some of it has subtle messages about MJ s feeling towards the press and also the obvious message of drugs are bad m kay   Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring  Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him   The actual feature film bit when it finally starts is only on for  

- Train/Test dataset 분리

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    df.review, df.sentiment, stratify=df.sentiment, random_state=2022
)
y_train.value_counts()

0    9375
1    9375
Name: sentiment, dtype: int64

- CountVectorizer로 변환

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(stop_words='english')

In [11]:
cvect.fit(X_train, y_train)
X_train_cv = cvect.transform(X_train)
X_train_cv.shape

(18750, 65213)

In [12]:
# test dataset은 train dataset을 변환한 Vectorizer로 변환해야 함
X_test_cv = cvect.transform(X_test)
X_test_cv.shape
# 뒤에 있는 단어의 집합이 같아짐!


(6250, 65213)

In [15]:
import time
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2022)
%time rfc.fit(X_train_cv, y_train)
rfc.score(X_test_cv, y_test)

CPU times: user 43.3 s, sys: 107 ms, total: 43.4 s
Wall time: 43.3 s


0.85088

In [17]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
%time nb.fit(X_train_cv, y_train)
nb.score(X_test_cv, y_test)

CPU times: user 30.7 ms, sys: 1.99 ms, total: 32.7 ms
Wall time: 43.2 ms


0.85504

ngram_range(1,2)

In [22]:
cvect2 = CountVectorizer(ngram_range=(1,2), stop_words='english')
cvect2.fit(X_train)
X_train_cv2 = cvect2.transform(X_train)
print(X_train_cv2.shape)

(18750, 1386558)


In [24]:
nb2 = MultinomialNB()
%time nb2.fit(X_train_cv2, y_train)
X_test_cv2= cvect2.transform(X_test)
nb2.score(X_test_cv2, y_test)

CPU times: user 216 ms, sys: 974 µs, total: 217 ms
Wall time: 285 ms


0.8672

- 모델 저장하고 물러오기

In [25]:
import joblib
joblib.dump(cvect2, 'imdb_cvect12.pkl')
joblib.dump(nb2, 'imdb_nb2.pkl')

['imdb_nb2.pkl']

In [ ]:
# VS코드
# joblib.dump(cvect2, 'model/imdb_cvect12.pkl')
# joblib.dump(nb2, 'model/imdb_nb2.pkl')
# git ignore에 'model/' 추가
# 바이너리 파일은 안올라가도록 세팅!

In [26]:
!ls -l

total 114376
-rw-r--r-- 1 root root 39184702 Apr 28 02:40 imdb_cvect12.pkl
-rw-r--r-- 1 root root 44370573 Apr 28 02:40 imdb_nb2.pkl
-rw-r--r-- 1 root root 33556378 Apr 28 01:35 labeledTrainData.tsv
drwxr-xr-x 1 root root     4096 Apr 25 13:46 sample_data


In [27]:
new_cvect = joblib.load('imdb_cvect12.pkl')
new_nb = joblib.load('imdb_nb2.pkl')

In [31]:
review = '''
Gotham City is rocked by a series of murders of key political figures, by a sadistic killer calling himself The Riddler (Paul Dano), who holds the city to ransom, revealing darkly guarded secrets and threatening more mayhem until he is stopped. 'The Batman' (Robert Pattinson), the alter ego of millionaire Bruce Wayne, starts an investigation of his own, crossing paths with a woman called Selina Kyle (Zoe Kravitz), who operates as 'Catwoman' and wants to find out what happened to her friend. It sets them both on a collision course with the darkest elements of Gotham's underworld, including notorious gangster 'The Penguin' (an unrecognisable Colin Farrell) and crime boss Carmine Falcone (John Turturro), who may be sitting on a devastating secret that could shatter both their lives.

The Batman series must be one of the most enduringly successful franchises in the world of comic book films, appearing in various different incarnations since its original creation. Here, writer/director Matt Reeves delivers a mammoth three hour long interpretation of his own of the character that many seem to be hailing as the best ever. Less doesn't seem to be more anymore, at least not to the cinema artistes, but from this end, all I can see is a film drowning in its own indulgence.

Reeves is obviously keen to maintain the bleak, heavy tone established by Christopher Nolan, and so we have a Gotham City with seemingly endless heavy rain, and gloomy skies, set against Pattinson's dark, brooding stare and gravelly voiceovers, set to the background of the emo rock soundtrack. It's just one of many things ripe for parody in a film that takes itself much too seriously and feels almost devoid of any humour and light. Even more overdone is Dano's dark 'Riddler', operating in some garish gimp suit that just looks ridiculous. Thrown in to this is the topical 'social justice' speeches that litter film and TV today, with Kravitz's Catwoman even praising the 'sinister' villain (who we're meant to hate!) at one point, for killing 'all these white privileged a$$holes.' It's just trying to cram a whole load of eggs in to one basket, over ambitious and overlong, and by the time of the main villain's grand 'master plan' at the end, you're too overwhelmed (or bored) to care.

Everyone else can think what they like, but personally this is one of the most pretentious Batman's ever, which might have been a bit more bearable if it had just been less self indulgent, or at least had an old fashioned, Lawrence of Arabia style 'intermission' at some point.
'''

In [32]:
import re
review = re.sub('[^a-zA-Z]', ' ', review).strip()
review

'Gotham City is rocked by a series of murders of key political figures  by a sadistic killer calling himself The Riddler  Paul Dano   who holds the city to ransom  revealing darkly guarded secrets and threatening more mayhem until he is stopped   The Batman   Robert Pattinson   the alter ego of millionaire Bruce Wayne  starts an investigation of his own  crossing paths with a woman called Selina Kyle  Zoe Kravitz   who operates as  Catwoman  and wants to find out what happened to her friend  It sets them both on a collision course with the darkest elements of Gotham s underworld  including notorious gangster  The Penguin   an unrecognisable Colin Farrell  and crime boss Carmine Falcone  John Turturro   who may be sitting on a devastating secret that could shatter both their lives   The Batman series must be one of the most enduringly successful franchises in the world of comic book films  appearing in various different incarnations since its original creation  Here  writer director Mat

In [35]:
review_cv = new_cvect.transform([review])
review_cv.shape

(1, 1386558)

In [36]:
new_nb.predict(review_cv)

array([1])